In [5]:
import pandas as pd
import numpy as np
data=pd.read_csv('apple_dataset/apple_quality.csv')
# drop the first colum
data=data.drop('A_id',axis=1)
#drop last row
data=data.drop(data.index[-1])
#all the columns but the last one is input 
X=data.iloc[:,:-1]
#the last column is the output
y=data.iloc[:,-1]

X=X.to_numpy()
y=y.to_numpy()
y[y=='good']=1
y[y=='bad']=0


### Add noise to check if the automatic relevance determination is able to find only the relevant features to predict the target.

In [6]:
# concatenate another column of random number to the input usgin gaussian distribution
X=np.concatenate((X,np.random.normal(size=(X.shape[0],1))),axis=1)

In [7]:
from tqdm import tqdm
import torch
import gpytorch
from gpytorch.models import ApproximateGP
from torch.utils.data import TensorDataset, DataLoader
class GPModel(ApproximateGP):
    def __init__(self, inducing_points, ard=False):

        if ard:
            ard_num_dim=inducing_points.size(-1)
        else:
            ard_num_dim=None


        variational_distribution = gpytorch.variational.CholeskyVariationalDistribution(inducing_points.size(-2))
        variational_strategy = gpytorch.variational.VariationalStrategy(self, inducing_points, variational_distribution, learn_inducing_locations=True)

        super().__init__(variational_strategy)

        self.mean_module = gpytorch.means.ZeroMean()

        # If you want to use different hyperparameters for each task,
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.MaternKernel(ard_num_dims=ard_num_dim, nu=2.5))
    def forward(self, x):
        # The forward function should be written as if we were dealing with each output
        # dimension in batch
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
        
#%%
class Classifier():

    def __init__(self, X, y, num_inducing=50):
        #pick the inducing from the training set 
        X=X.astype(float)
        y=y.astype(float)
        self.X = torch.from_numpy(X)
        self.Y = torch.from_numpy(y)
        self.inducing_points = self.X[:num_inducing]
        self.create_model()
        self.move_model_to_cuda()
        
    def create_model(self):        
        self.model = GPModel( inducing_points=self.inducing_points, ard=True)
        self.likelihood = gpytorch.likelihoods.BernoulliLikelihood()

    def move_model_to_cuda(self):
        self.model=self.model.cuda()
        self.likelihood=self.likelihood.cuda()
        self.inducing_points=self.inducing_points.cuda()
        self.X=self.X.cuda()
        self.Y=self.Y.cuda()
        self.model=self.model.double()
        self.likelihood=self.likelihood.double()
        self.inducing_points=self.inducing_points.double()

    def train(self, num_epochs=3):
        self.model.train()
        self.likelihood.train()

        optimizer = torch.optim.Adam([
            {'params': self.model.parameters()},
            {'params': self.likelihood.parameters()},
        ], lr=0.01)

        # Our loss object. We're using the VariationalELBO
        mll = gpytorch.mlls.VariationalELBO(self.likelihood, self.model, num_data=self.Y.size(-1))

        train_dataset = TensorDataset(self.X, self.Y)
        self.train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)

        # epochs_iter = tqdm.notebook.tqdm(range(num_epochs), desc="Epoch")
        epochs_iter = tqdm(range(num_epochs))
        for i in epochs_iter:
            # Within each iteration, we will go over each minibatch of data
            minibatch_iter = tqdm(self.train_loader, desc="Minibatch", leave=False)
            for x_batch, y_batch in minibatch_iter:
                optimizer.zero_grad()
                output = self.model(x_batch)
                loss = -mll(output, y_batch)
                minibatch_iter.set_postfix(loss=loss.item())
                loss.backward()
                optimizer.step()
        print("Training finished")
        print("Lengthscale: ")        
        print(self.model.covar_module.base_kernel.lengthscale)
        print("Outputscale: ")
        print(torch.sqrt(self.model.covar_module.outputscale))
    
    def predict(self, X):
        self.model.eval()
        self.likelihood.eval()
        X=X.astype(float)
        X = torch.from_numpy(X).cuda().double()
        with torch.no_grad(), gpytorch.settings.fast_pred_var():
            observed_pred = self.likelihood(self.model(X))
            return observed_pred.mean.cpu().numpy()
        

In [8]:
# slip the data into training and testing set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

svgp=Classifier(num_inducing=200, X=X_train, y=y_train)

svgp.train(num_epochs=20) 

y_pred=svgp.predict(X_test)
#compute accuracy
y_pred[y_pred>0.5]=1
y_pred[y_pred<=0.5]=0
accuracy=(y_pred==y_test).sum()/len(y_test)
print('accuracy:',accuracy)

Minibatch:   1%|▏         | 4/320 [00:00<00:07, 39.94it/s, loss=0.902]

100%|██████████| 20/20 [01:36<00:00,  4.82s/it]


Training finished
Lengthscale: 
tensor([[ 4.6167,  5.4072,  5.3381,  4.7884,  5.3512,  3.0080,  6.3960, 16.1792]],
       device='cuda:0', dtype=torch.float64, grad_fn=<SoftplusBackward0>)
Outputscale: 
tensor(1.6097, device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward0>)
accuracy: 0.90375
